<a href="https://colab.research.google.com/github/nawon0719/KU-NLP-Sentiment-Analysis/blob/main/Korean_Sentiment_Analysis_Submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**준비 사항**

In [ ]:
!pip install transformers
!pip install torch

     |████████████████████████████████| 1.4MB 13.1MB/s 
     |████████████████████████████████| 890kB 53.5MB/s 
     |████████████████████████████████| 2.9MB 50.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=54c9ca4d114f1aff37d2e96ec1b9b6b0d4a7ec3e7ee75e127fdaf166f6cfb967
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import tensorflow as tf
import torch
import transformers

from transformers import ElectraForSequenceClassification, ElectraModel,ElectraTokenizer,ElectraConfig
from transformers import get_linear_schedule_with_warmup, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

#**Train set 로드**

In [ ]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 6.59 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [ ]:
# 데이터 로드
train = pd.read_csv("nsmc/ratings_train.txt", sep='\t')

label이 0이면 부정, label이 1이면 긍정

In [ ]:
print(train.index)
train.info()
train['label'].value_counts()

RangeIndex(start=0, stop=150000, step=1)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


0    75173
1    74827
Name: label, dtype: int64

#전처리

##**맞춤법 검사**
시간효율상 맞춤법 검사 완료, 수정 후 저장한 train파일 유첨 (train_spell.csv)

하단 3개 셀 skip하여도 무방

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git
!pip install tqdm

from tqdm.notebook import tqdm
from hanspell import spell_checker

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-e7_hkru4
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-e7_hkru4
  Created wheel for py-hanspell: filename=py_hanspell-1.1-cp36-none-any.whl size=4854 sha256=adfb9c8a31c10e87975fe245c9b571a0f97f8f59a3a20f9384c0e936c95e29b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-dl1hsqe0/wheels/0a/25/d1/e5e96476dbb1c318cc26c992dd493394fe42b0c204b3e65588
Successfully built py-hanspell


In [ ]:
#학습데이터 15만개 맞춤법 검사
for i in tqdm(range(len(train))):
  try:
    result_train = spell_checker.check(train['document'][i])
    train['document'][i] = result_train.as_dict()['checked']
  except:
    pass

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
#맞춤법 검사 완료된 데이터로 로드
train = pd.read_csv("train_spell.csv", sep=',',encoding='utf-8-sig')
train

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠... 포스터 보고 초등학생영화 줄.... 오버 연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 .. 솔직히 재미는 없다.. 평점 조정,0
4,6483659,사이먼 페그의 익살스러운 연기가 돋보였던 영화! 스파이더맨에서 늙어 보이기만 했던 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


##데이터 인풋

In [ ]:
MAX_LEN = 128

def getInputs(dataset):
  data = dataset.copy(deep=True)

  if 'document' in data.columns:
    sentences = data['document']
  else:
    sentences = data['Sentence']

  #토크나이저에 적합한 입력 방식으로 변경
  sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
  
  tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator", do_lower_case=False)
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  attention_masks = []
  for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
      attention_masks.append(seq_mask)

  return input_ids, attention_masks

In [ ]:
#데이터를 토큰+패딩, 어텐션마스크로 분리
rating_inputs, rating_masks = getInputs(train)
labels = train['label'].values

# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(rating_inputs, 
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(rating_masks, 
                                                       rating_inputs, 
                                                       random_state=2018, 
                                                       test_size=0.1)

In [ ]:
batch_size = 32

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)

validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)	

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

#**모델 생성**

In [ ]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [ ]:
# ELECTRA 모델 생성

model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels = 2)
model.cuda()

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

#학습

In [ ]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 3e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

epochs = 5

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
# Accuracy
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
#Time
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
seed_val = 1211
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
#Training Section
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_loss = 0

    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        batch = tuple(t.to(device) for t in batch)
        
        b_input_ids, b_input_mask, b_labels = batch
            
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        loss = outputs[0]

        total_loss += loss.item()

        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.5f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
### Validation Section ### 

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
      
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():    
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로직과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.5f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 5 ========
Training...
  Batch   500  of  4,219.    Elapsed: 0:03:17.
  Batch 1,000  of  4,219.    Elapsed: 0:06:35.
  Batch 1,500  of  4,219.    Elapsed: 0:09:52.
  Batch 2,000  of  4,219.    Elapsed: 0:13:09.
  Batch 2,500  of  4,219.    Elapsed: 0:16:27.
  Batch 3,000  of  4,219.    Elapsed: 0:19:44.
  Batch 3,500  of  4,219.    Elapsed: 0:23:02.
  Batch 4,000  of  4,219.    Elapsed: 0:26:19.

  Average training loss: 0.38936
  Training epcoh took: 0:27:45

Running Validation...
  Accuracy: 0.85446
  Validation took: 0:00:58

======== Epoch 2 / 5 ========
Training...
  Batch   500  of  4,219.    Elapsed: 0:03:17.
  Batch 1,000  of  4,219.    Elapsed: 0:06:35.
  Batch 1,500  of  4,219.    Elapsed: 0:09:52.
  Batch 2,000  of  4,219.    Elapsed: 0:13:10.
  Batch 2,500  of  4,219.    Elapsed: 0:16:27.
  Batch 3,000  of  4,219.    Elapsed: 0:19:44.
  Batch 3,500  of  4,219.    Elapsed: 0:23:02.
  Batch 4,000  of  4,219.    Elapsed: 0:26:19.

  Average training loss: 0

In [ ]:
logits[0:100]

array([[ 2.8634276, -2.9241   ]], dtype=float32)

#**test set 평가**

Test Data Preprocessing

In [ ]:
#데이터 읽어오기
test = pd.read_csv("/content/ko_data.csv",encoding='CP949')
print(test)

In [ ]:
def getIndex(dataset):
  data = dataset.copy(deep = True)
  input_index = data.index.tolist()
  return torch.tensor(input_index)

In [ ]:
#맞춤법검사
for i in tqdm(range(len(test))):
  try:
    result_test = spell_checker.check(test['Sentence'][i])
    test['Sentence'][i] = result_test.as_dict()['checked']
  except:
    pass

#인풋데이터화
test_inputs, test_masks = getInputs(test)

test_index = getIndex(test)
test_inputs = torch.tensor(test_inputs) #sentence
test_masks = torch.tensor(test_masks) 

batch_size = 32

test_data = TensorDataset(test_index, test_inputs, test_masks)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=1)

RangeIndex(start=0, stop=11187, step=1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


Test data prediction

In [ ]:
test_result = test.copy(deep = True)
test_result['Predicted'] = 'default'
classes = [0, 1]

In [ ]:
t0 = time.time()

model.eval()

nb_eval_steps, nb_eval_examples = 0, 0

for step, batch in enumerate(test_dataloader):
    if step % 1000 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    batch = tuple(t.to(device) for t in batch)
    
    b_index, b_input_ids, b_input_mask = batch
    
    with torch.no_grad():     
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    idx = b_index.item()
    test_result['Predicted'][idx] = classes[np.argmax(logits)]
    
    nb_eval_steps += 1

print("")
print("Test took: {:}".format(format_time(time.time() - t0)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


  Batch 1,000  of  11,187.    Elapsed: 0:00:13.
  Batch 2,000  of  11,187.    Elapsed: 0:00:26.
  Batch 3,000  of  11,187.    Elapsed: 0:00:39.
  Batch 4,000  of  11,187.    Elapsed: 0:00:53.
  Batch 5,000  of  11,187.    Elapsed: 0:01:06.
  Batch 6,000  of  11,187.    Elapsed: 0:01:19.
  Batch 7,000  of  11,187.    Elapsed: 0:01:32.
  Batch 8,000  of  11,187.    Elapsed: 0:01:45.
  Batch 9,000  of  11,187.    Elapsed: 0:01:58.
  Batch 10,000  of  11,187.    Elapsed: 0:02:12.
  Batch 11,000  of  11,187.    Elapsed: 0:02:25.

Test took: 0:02:27


In [ ]:
test_result['Predicted']

0        1
1        0
2        1
3        1
4        0
        ..
11182    1
11183    0
11184    1
11185    1
11186    0
Name: Predicted, Length: 11187, dtype: object

In [ ]:
submission=test_result.drop(columns=['Sentence'])
submission_csv=submission.to_csv('test_speller.csv',index=False)
from google.colab import files

files.download('test_speller.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

hanspell 코드 참고 : https://github.com/ssut/py-hanspell

koElectra 코드 활용 : https://github.com/monologg/KoELECTRA

전체적인 모델링 코드 및 학습내용 참고 : 

1) https://mccormickml.com/2019/07/22/BERT-fine-tuning/

2) https://colab.research.google.com/drive/1tIf0Ugdqg4qT7gcxia3tL7und64Rv1dP

##새로운문장TEST

In [ ]:
# 입력 데이터 변환
def convert_input_data(sentences):

  #for sentences in sentences:
    try:
      result_test = spell_checker.check(sentences)
      sentences = result_test.as_dict()['checked']
    except:
      pass

    
    #토크나이저로 문장을 토큰으로 분리
    tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator", do_lower_case=False) #KoELECTRA 토크나이저 위치
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    MAX_LEN = 128

    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []
    
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

    # 문장 테스트
def test_sentences(sentences):

    model.eval()

    inputs, masks = convert_input_data(sentences)

    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()

    return logits

In [ ]:
logits=test_sentences(['재미없음'])

print(logits)
print(np.argmax(logits))

[[-0.3344997   0.36547226]]
1


In [ ]:
logits=test_sentences(['재미 없음'])
print(logits)
print(np.argmax(logits))

[[ 2.8634276 -2.9241   ]]
0
